# Case Study

The goal is to analyze the best places to be a digital nomad, taking into account the different types of marine species we can see in each of them and how many deaths and the cause of the death by city.

Four datasets were imported:

- First one is a database where we have different variables such as the cost of been a nomad, the security of the country, the punctuation of the city of nomads that have been living there, internet speed, among others.

- Second, we have a database with all the cities and countries of the world to combine it with the first database.

- Third, we have a database including all the marine species we can see in each country, as we only have the scientific name, we are going to do a count of the variable per city.

- Last one, is a database are the deaths that have occurred during the dive.

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ssl
from geopy.geocoders import Nominatim
import geocoder 

# Working with data

Upload the first database. In this database we can see a series of countries and a series of values, such as the cost of living, the cost of a coworking, etc.. so that the person according to their criteria can assess the country that best suits him/her. 

In [2]:
data = pd.read_csv('../Data/raw/nomad_cities.csv', sep='\t')

#find this database in: https://www.kaggle.com/datasets/danielhain/digital-nomads

In [3]:
data.head()

,Unnamed: 0,coffee_in_cafe,cost_beer,cost_coworking,cost_expat,cost_nomad,female_friendly,fragile_states_index,free_wifi_available,freedom_score,...,nightlife,nomadScore,nomad_score,peace_score,place,places_to_work,press_freedom_index,racism,safety,weed
0,0,1.73,1.73,152.41,1273,1364,1.0,52.7,0.4,0.6,...,1.0,1.00,1.00,0.8,Budapest,1.0,28.17,0.40,0.60,0
1,1,0.85,0.85,98.88,780,777,0.8,78.8,0.6,0.2,...,0.4,0.95,0.95,0.4,Chiang Mai,0.8,44.53,0.40,0.80,0
2,2,1.99,1.99,159.13,1653,1639,1.0,40.8,0.6,0.8,...,1.0,0.94,0.94,0.8,Prague,1.0,16.66,0.42,0.80,1
3,3,1.88,1.88,47.01,1640,1545,1.0,DotMap(__next__=DotMap()),1.0,0.6,...,0.6,0.94,0.94,DotMap(__next__=DotMap()),Taipei,1.0,24.37,0.00,1.00,0
4,4,5.00,5.00,200.00,3309,3028,0.8,34,0.6,0.6,...,1.0,0.94,0.94,0.8,Austin,1.0,22.49,0.80,0.73,0


In [4]:
data.columns

Index(['Unnamed: 0', 'coffee_in_cafe', 'cost_beer', 'cost_coworking',
       'cost_expat', 'cost_nomad', 'female_friendly', 'fragile_states_index',
       'free_wifi_available', 'freedom_score', 'friendly_to_foreigners',
       'internet_speed', 'latitude', 'leisure', 'lgbt_friendly', 'life_score',
       'longitude', 'nightlife', 'nomadScore', 'nomad_score', 'peace_score',
       'place', 'places_to_work', 'press_freedom_index', 'racism', 'safety',
       'weed'],
      dtype='object')

Drop the columns that are not important or that don't gives the info you need

In [5]:
data = data.drop(['longitude', 'latitude', 'Unnamed: 0', 'cost_expat', 'leisure', 'lgbt_friendly', 'nomadScore', 'press_freedom_index', 'weed', 'peace_score'], axis=1)
data.head()

,coffee_in_cafe,cost_beer,cost_coworking,cost_nomad,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,internet_speed,life_score,nightlife,nomad_score,place,places_to_work,racism,safety
0,1.73,1.73,152.41,1364,1.0,52.7,0.4,0.6,0.6,31,0.86,1.0,1.00,Budapest,1.0,0.40,0.60
1,0.85,0.85,98.88,777,0.8,78.8,0.6,0.2,0.6,14,0.75,0.4,0.95,Chiang Mai,0.8,0.40,0.80
2,1.99,1.99,159.13,1639,1.0,40.8,0.6,0.8,0.8,15,0.83,1.0,0.94,Prague,1.0,0.42,0.80
3,1.88,1.88,47.01,1545,1.0,DotMap(__next__=DotMap()),1.0,0.6,0.8,16,0.93,0.6,0.94,Taipei,1.0,0.00,1.00
4,5.00,5.00,200.00,3028,0.8,34,0.6,0.6,0.8,118,0.95,1.0,0.94,Austin,1.0,0.80,0.73


Check for NaN values

In [6]:
data.isna().sum()

coffee_in_cafe            0
cost_beer                 0
cost_coworking            0
cost_nomad                0
female_friendly           0
fragile_states_index      0
free_wifi_available       0
freedom_score             0
friendly_to_foreigners    0
internet_speed            0
life_score                0
nightlife                 0
nomad_score               0
place                     0
places_to_work            0
racism                    0
safety                    0
dtype: int64

Check the data type to know if you need the change the type, and which one could be the cause for not been the correct type, for example, values that don't proceed

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   coffee_in_cafe          781 non-null    float64
 1   cost_beer               781 non-null    float64
 2   cost_coworking          781 non-null    float64
 3   cost_nomad              781 non-null    int64  
 4   female_friendly         781 non-null    float64
 5   fragile_states_index    781 non-null    object 
 6   free_wifi_available     781 non-null    float64
 7   freedom_score           781 non-null    object 
 8   friendly_to_foreigners  781 non-null    float64
 9   internet_speed          781 non-null    int64  
 10  life_score              781 non-null    float64
 11  nightlife               781 non-null    float64
 12  nomad_score             781 non-null    float64
 13  place                   781 non-null    object 
 14  places_to_work          781 non-null    fl

In [8]:
data['fragile_states_index'].unique()

array(['52.7', '78.8', '40.8', 'DotMap(__next__=DotMap())', '34', '39.8',
       '29.2', '29', '21.3', '22.6', '74.9', '48.4', '23.8', '28.6',
       '70.7', 'DotMap()', '22.5', '36.1', '28.2', '21.8', '39.6', '65.3',
       '34.5', '43.1', '27.5', '32.4', '55.9', '53.2', '35.1', '52.9',
       '21.2', '70.4', '32.9', '40.7', '47.4', '43.4', '36.2', '75.5',
       '67', '73.9', '81', '79.6', '41.9', '72', '21.5', '69.6', '44.9',
       '77.3', '18.8', '64', '45.1', '52.4', '33.9', '53.7', '78.9', '78',
       '42.4', '66.1', '70.8', '65', '44.5', '51.6', '75.6', '74.2',
       '66.3', '80.2', '62', '24.1', '72.6', '87.7', '55.2', '83.6',
       '84.7', '43.2', '61.2', '63.4', '22.8', '72.2', '76.3', '66', '74',
       '73.2', '69.9', '90.7', '66.5', '87.4', '98.3', '89.6', '78.5',
       '101.7', '79', '72.5', '90.2', '56.6', '83.2', '96.3', '84.4',
       '103.5', '74.6', '71.1', '95.2', '91.2', '78.3', '91.3', '83.5',
       '87.8', '84.2', '81.8', '97.7', '71.2', '79.8', '83.8', '97

In [9]:
data['fragile_states_index'] = data['fragile_states_index'].replace('DotMap(__next__=DotMap())', '')

In [10]:
data['freedom_score'].unique()

array(['0.6', '0.2', '0.8', '1', '0.4', 'DotMap()'], dtype=object)

In [11]:
data['freedom_score'] = data['freedom_score'].replace('DotMap()', '')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   coffee_in_cafe          781 non-null    float64
 1   cost_beer               781 non-null    float64
 2   cost_coworking          781 non-null    float64
 3   cost_nomad              781 non-null    int64  
 4   female_friendly         781 non-null    float64
 5   fragile_states_index    781 non-null    object 
 6   free_wifi_available     781 non-null    float64
 7   freedom_score           781 non-null    object 
 8   friendly_to_foreigners  781 non-null    float64
 9   internet_speed          781 non-null    int64  
 10  life_score              781 non-null    float64
 11  nightlife               781 non-null    float64
 12  nomad_score             781 non-null    float64
 13  place                   781 non-null    object 
 14  places_to_work          781 non-null    fl

Change the type to the correct one

In [13]:
data['fragile_states_index'] = pd.to_numeric(data['fragile_states_index'], errors='coerce')
data['freedom_score'] = pd.to_numeric(data['freedom_score'], errors='coerce')

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 781 entries, 0 to 780
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   coffee_in_cafe          781 non-null    float64
 1   cost_beer               781 non-null    float64
 2   cost_coworking          781 non-null    float64
 3   cost_nomad              781 non-null    int64  
 4   female_friendly         781 non-null    float64
 5   fragile_states_index    756 non-null    float64
 6   free_wifi_available     781 non-null    float64
 7   freedom_score           769 non-null    float64
 8   friendly_to_foreigners  781 non-null    float64
 9   internet_speed          781 non-null    int64  
 10  life_score              781 non-null    float64
 11  nightlife               781 non-null    float64
 12  nomad_score             781 non-null    float64
 13  place                   781 non-null    object 
 14  places_to_work          781 non-null    fl

Replace the NaN values with the mean of the variables that have a relationship with it

In [15]:
avg_cols = ['female_friendly', 'friendly_to_foreigners', 'life_score', 'racism', 'safety']
avg = data[avg_cols].mean(axis=1)
avg_index = avg*100


In [16]:
data['fragile_states_index'] = data['fragile_states_index'].fillna(avg_index)

In [17]:
data['freedom_score'] = data['freedom_score'].fillna(avg)

In [18]:
data.isna().sum()

coffee_in_cafe            0
cost_beer                 0
cost_coworking            0
cost_nomad                0
female_friendly           0
fragile_states_index      0
free_wifi_available       0
freedom_score             0
friendly_to_foreigners    0
internet_speed            0
life_score                0
nightlife                 0
nomad_score               0
place                     0
places_to_work            0
racism                    0
safety                    0
dtype: int64

In [19]:
data = data.rename(columns={"place": "city"})

In [20]:
copy_data = data.copy()

Upload the second database, this database is needed to find the different countries for the cities of the first database

In [21]:
country = pd.read_csv('../../../../../Downloads/simplemaps_worldcities_basicv1.76/worldcities.csv')
country

#finde the database in: https://simplemaps.com/data/world-cities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629
...,...,...,...,...,...,...,...,...,...,...,...
44686,Numto,Numto,63.6667,71.3333,Russia,RU,RUS,Khanty-Mansiyskiy Avtonomnyy Okrug-Yugra,NaN,10.0,1643985006
44687,Nord,Nord,81.7166,-17.8000,Greenland,GL,GRL,NaN,NaN,10.0,1304217709
44688,Timmiarmiut,Timmiarmiut,62.5333,-42.2167,Greenland,GL,GRL,Kujalleq,NaN,10.0,1304206491
44689,San Rafael,San Rafael,-16.7795,-60.6799,Bolivia,BO,BOL,Santa Cruz,NaN,NaN,1068405265


In [22]:
country.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


merge both databases and select the columns you want to have in the merged database

In [23]:
merged_data = pd.merge(copy_data, country, on="city")

In [24]:
merged_data.columns

Index(['coffee_in_cafe', 'cost_beer', 'cost_coworking', 'cost_nomad',
       'female_friendly', 'fragile_states_index', 'free_wifi_available',
       'freedom_score', 'friendly_to_foreigners', 'internet_speed',
       'life_score', 'nightlife', 'nomad_score', 'city', 'places_to_work',
       'racism', 'safety', 'city_ascii', 'lat', 'lng', 'country', 'iso2',
       'iso3', 'admin_name', 'capital', 'population', 'id'],
      dtype='object')

In [25]:
nomad = merged_data[['country', 'city', 'coffee_in_cafe', 'cost_beer', 'cost_coworking', 'cost_nomad',
       'female_friendly', 'fragile_states_index', 'free_wifi_available',
       'freedom_score', 'friendly_to_foreigners', 'internet_speed',
       'life_score', 'nightlife', 'nomad_score', 'places_to_work', 'racism', 'safety']]

In [26]:
nomad

,country,city,coffee_in_cafe,cost_beer,cost_coworking,cost_nomad,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,internet_speed,life_score,nightlife,nomad_score,places_to_work,racism,safety
0,Hungary,Budapest,1.73,1.73,152.41,1364,1.0,52.7,0.4,0.6,0.6,31,0.86,1.0,1.00,1.0,0.40,0.60
1,Thailand,Chiang Mai,0.85,0.85,98.88,777,0.8,78.8,0.6,0.2,0.6,14,0.75,0.4,0.95,0.8,0.40,0.80
2,Czechia,Prague,1.99,1.99,159.13,1639,1.0,40.8,0.6,0.8,0.8,15,0.83,1.0,0.94,1.0,0.42,0.80
3,Taiwan,Taipei,1.88,1.88,47.01,1545,1.0,74.6,1.0,0.6,0.8,16,0.93,0.6,0.94,1.0,0.00,1.00
4,United States,Austin,5.00,5.00,200.00,3028,0.8,34.0,0.6,0.6,0.8,118,0.95,1.0,0.94,1.0,0.80,0.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1074,United States,Damascus,2.65,2.65,215.73,1162,0.2,110.8,0.2,0.2,0.2,2,0.01,0.2,0.19,0.4,0.00,0.20
1075,Pakistan,Bahawalpur,4.00,4.00,28.58,2637,0.2,101.7,0.4,0.2,0.4,2,0.11,0.2,0.18,0.4,0.20,0.20
1076,Iraq,Baghdad,2.00,2.00,183.00,2980,0.2,104.7,0.4,0.2,0.4,3,0.09,0.2,0.17,0.4,0.60,0.20
1077,Colombia,Cali,1.60,1.60,32.01,1099,0.4,80.2,0.6,0.2,1.0,4,0.36,1.0,0.16,0.4,0.40,0.20


In [27]:
nomad.isna().sum()

country                   0
city                      0
coffee_in_cafe            0
cost_beer                 0
cost_coworking            0
cost_nomad                0
female_friendly           0
fragile_states_index      0
free_wifi_available       0
freedom_score             0
friendly_to_foreigners    0
internet_speed            0
life_score                0
nightlife                 0
nomad_score               0
places_to_work            0
racism                    0
safety                    0
dtype: int64

In [28]:
nomad['country'].unique()

array(['Hungary', 'Thailand', 'Czechia', 'Taiwan', 'United States',
       'Spain', 'Colombia', 'Philippines', 'Australia', 'Venezuela',
       'Portugal', 'Brazil', 'Belgium', 'New Zealand', 'United Kingdom',
       'India', 'Sweden', 'Indonesia', 'Argentina', 'Costa Rica',
       'Germany', 'Paraguay', 'Chile', 'Vietnam', 'Hong Kong',
       'South Korea', 'Netherlands', 'Canada', 'Switzerland', 'Malta',
       'Suriname', 'Ecuador', 'Trinidad and Tobago', 'France', 'Italy',
       'Austria', 'Greece', 'Panama', 'Japan', 'Norway', 'Mexico',
       'Singapore', 'Poland', 'Latvia', 'Estonia', 'Uruguay', 'Macau',
       'Macedonia', 'China', 'Ireland', 'Seychelles', 'El Salvador',
       'Belarus', 'Russia', 'Dominican Republic', 'Peru', 'Serbia',
       'Denmark', 'Armenia', 'Slovakia', 'Turkey', 'Finland', 'Cyprus',
       'Croatia', 'Slovenia', 'Puerto Rico', 'Honduras', 'Bulgaria',
       'Gibraltar', 'Ukraine', 'Georgia', 'Jordan', 'Lithuania',
       'Sierra Leone', 'Malaysia', 'C

In [29]:
nomad = nomad.to_csv('../Data/cleaned/nomad_cities.csv', index=False)

Upload the third database

In [30]:
data3 = pd.read_csv('../Data/raw/points_data.csv')

#finde the database in: https://www.iucnredlist.org/search?dl=true&permalink=0bc75044-18cd-44db-8ed4-88da4464cdbe

In [31]:
data3.head()

,assessment_id,id_no,sci_name,presence,origin,seasonal,compiler,year,citation,legend,subspecies,subpop,dist_comm,island,tax_comm,source,basisofrec,event_year,longitude,latitude
0,719371,154767,Membras martinica,1,1,1,T. Contreras McBeath,2018,IUCN Freshwater Biodiversity Unit,Extant (resident),,,,,,T. Contreras McBeath 2014,,NaN,-99.3900,26.772500
1,719371,154767,Membras martinica,1,1,1,T. Contreras McBeath,2018,IUCN Freshwater Biodiversity Unit,Extant (resident),,,,,,T. Contreras McBeath 2014,,NaN,-99.1950,26.056667
2,719371,154767,Membras martinica,1,1,1,T. Contreras McBeath,2018,IUCN Freshwater Biodiversity Unit,Extant (resident),,,,,,T. Contreras McBeath 2014,,NaN,-99.1950,26.065278
3,719371,154767,Membras martinica,1,1,1,T. Contreras McBeath,2018,IUCN Freshwater Biodiversity Unit,Extant (resident),,,,,,T. Contreras McBeath 2014,,NaN,-99.0075,26.143889
4,719371,154767,Membras martinica,1,1,1,T. Contreras McBeath,2018,IUCN Freshwater Biodiversity Unit,Extant (resident),,,,,,T. Contreras McBeath 2014,,NaN,-98.9625,26.191944


Check for nan values and drop columns you don't need

In [32]:
data3.isna().sum()

assessment_id        0
id_no                0
sci_name             0
presence             0
origin               0
seasonal             0
compiler             0
year                 0
citation             0
legend               0
subspecies        1529
subpop            1699
dist_comm         2496
island            1702
tax_comm          2753
source             804
basisofrec        1391
event_year       19457
longitude            0
latitude             0
dtype: int64

In [33]:
data3 = data3.drop(['event_year', 'source', 'basisofrec', 'citation', 'legend', 'assessment_id', 'id_no', 'year', 'compiler', 'subspecies', 'dist_comm', 'island', 'tax_comm', 'subpop'], axis=1)
data3.head()

,sci_name,presence,origin,seasonal,longitude,latitude
0,Membras martinica,1,1,1,-99.3900,26.772500
1,Membras martinica,1,1,1,-99.1950,26.056667
2,Membras martinica,1,1,1,-99.1950,26.065278
3,Membras martinica,1,1,1,-99.0075,26.143889
4,Membras martinica,1,1,1,-98.9625,26.191944


In [34]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22034 entries, 0 to 22033
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   sci_name   22034 non-null  object 
 1   presence   22034 non-null  int64  
 2   origin     22034 non-null  int64  
 3   seasonal   22034 non-null  int64  
 4   longitude  22034 non-null  float64
 5   latitude   22034 non-null  float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.0+ MB


In [35]:
data3.isna().sum()

sci_name     0
presence     0
origin       0
seasonal     0
longitude    0
latitude     0
dtype: int64

As in the first two databases country column is shown, but not here, the thing to do is with geocoder try to find the country referred to those latitude and longitude 

In [36]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [37]:
from geopy.geocoders import Nominatim
import geocoder 

geolocator = Nominatim(user_agent="my-custom-user-agent",timeout=3)

geolocator.reverse(str(data3.iloc[0,-1]) + ',' + str(data3.iloc[0,-2])).raw['address']['country']


'México'

In [38]:
data3['country'] = None

In [39]:
data3.columns

Index(['sci_name', 'presence', 'origin', 'seasonal', 'longitude', 'latitude',
       'country'],
      dtype='object')

In [40]:
import geocoder

def refine_country2(x, y):
    
    try:
        country = geolocator.reverse(str(x) + ',' + str(y)).raw['address']['country']
        print(country)
        return country
    except:
            print("Location not found!")
            return np.nan
    
data3["country"] = list(map(refine_country2, data3['latitude'], data3['longitude']))

México
México
México
México
México
México
México
México
México
México
United States
México
México
México
United States
United States
México
México
México
México
México
México
México
México
México
México
Guatemala
El Salvador
El Salvador
El Salvador
El Salvador
El Salvador
Honduras
Costa Rica
Costa Rica
Costa Rica
Costa Rica
Panamá
Panamá
Colombia
Location not found!
Colombia
France
Colombia
France
France
France
France
France
France
France
France
Tonga
Tonga
Sāmoa
United States
Kiribati
United States
United States
France
France
France
México
México
México
México
México
México
México
México
México
México
Location not found!
Location not found!
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
Location not found!
México
México
South Africa
South Africa
South Africa
South Africa
South Africa
South Africa


Location not found!
Location not found!
Location not found!
日本
Location not found!
日本
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Papua Niugini
Papua Niugini
Papua Niugini
Location not found!
New Zealand/Aotearoa
Location not found!
Location not found!
Location not found!
Location not found!
Cuba
Cuba
Cuba
Cuba
Cuba
Cuba
Cuba
Cuba
Cuba
Cuba
South Africa
Moçambique
Moçambique
Moçambique
Moçambique
Moçambique
Moçambique
India
India
India
India
India
Location not found!
ประเทศไทย
ประเทศไทย
ประเทศไทย
ประเทศไทย
ประเทศไทย
ประเทศไทย
မြန်မာ
ประเทศไทย
ประเทศไทย
Malaysia
Malaysia
ประเทศไทย
Malaysia
Malaysia
Singapore
Singapore
Singapore
Singapore
Việt Nam
Việt Nam
Việt Nam
Việt Nam
Việt Nam
Việt Nam
Việt Nam
Việt Nam
Việt Nam
Việt Nam
Việt Nam
Việt Nam
中国
中国
中国
中国
中国
中国
中国
Indonesia
中国
中国
中国
中国
中国
中国
中国
臺灣
臺灣
臺灣
中国
臺灣
臺灣
Philippines
臺灣
臺灣
中国


ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව 

Panamá
Panamá
Panamá
Panamá
Panamá
Panamá
Panamá
Cuba
Panamá
Panamá
Panamá
Panamá
Panamá
Panamá
Panamá
Panamá
Panamá
The Bahamas
The Bahamas
Panamá
Panamá
Panamá
Panamá
Jamaica
Jamaica
Jamaica
Colombia
Colombia
Colombia
Colombia
Cuba
Cuba
Cuba
Ayiti
Colombia
Ayiti
Colombia
Colombia
Colombia
Colombia
Colombia
Colombia
Colombia
Colombia
Ayiti
Colombia
Ayiti
Ayiti
Ayiti
Ayiti
República Dominicana
República Dominicana
República Dominicana
República Dominicana
República Dominicana
República Dominicana
República Dominicana
República Dominicana
República Dominicana
República Dominicana
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
Location not found!
United States
United States
United States
United States
Un

United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United

...

United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United

United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
United States
México
México
México
México
México
México
México

Indonesia
Philippines
Philippines
日本
日本
Indonesia
Philippines
Philippines
日本
Philippines
Philippines
Philippines
Philippines
Indonesia
日本
Indonesia
日本
Indonesia
Belau
Belau
Micronesia
Papua Niugini
United States
United States
United States
United States
United States
Papua Niugini
Papua Niugini
Solomon Islands
Solomon Islands
Solomon Islands
Solomon Islands
Ṃajeḷ
Ṃajeḷ
Vanuatu
Vanuatu
Vanuatu
Vanuatu
Ṃajeḷ
Naoero
Vanuatu
France
Ṃajeḷ
Ṃajeḷ
Vanuatu
Ṃajeḷ
Viti
Tuvalu
Tuvalu
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
Location not found!
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
Location not found!
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
México
Location not found!
Location not found!
Location not fou

Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!
Location not found!


ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව இலங்கை
ශ්‍රී ලංකාව 

Work with a copy as the cell above is very time consuming.

In [41]:
data3_copy = data3.copy()

In [42]:
data3_copy

,sci_name,presence,origin,seasonal,longitude,latitude,country
0,Membras martinica,1,1,1,-99.390000,26.772500,México
1,Membras martinica,1,1,1,-99.195000,26.056667,México
2,Membras martinica,1,1,1,-99.195000,26.065278,México
3,Membras martinica,1,1,1,-99.007500,26.143889,México
4,Membras martinica,1,1,1,-98.962500,26.191944,México
...,...,...,...,...,...,...,...
22029,Thermosipho auzendei,1,1,1,-112.032997,-31.850000,NaN
22030,Thermosipho auzendei,1,1,1,-111.917000,-31.150000,NaN
22031,Gigantopelta chessoia,1,1,1,-30.325001,-56.091702,NaN
22032,Gigantopelta chessoia,1,1,1,-29.930000,-60.049999,NaN


In [43]:
data3_copy.rename(columns={'sci_name': 'marine_species'}, inplace=True)

Once we have all the coountries, drop NaN values

In [44]:
data3_copy = data3_copy.dropna(subset=["country"])
data3_copy

,marine_species,presence,origin,seasonal,longitude,latitude,country
0,Membras martinica,1,1,1,-99.390000,26.772500,México
1,Membras martinica,1,1,1,-99.195000,26.056667,México
2,Membras martinica,1,1,1,-99.195000,26.065278,México
3,Membras martinica,1,1,1,-99.007500,26.143889,México
4,Membras martinica,1,1,1,-98.962500,26.191944,México
...,...,...,...,...,...,...,...
22010,Phymorhynchus oculatus,1,1,1,151.669998,-3.725000,Papua Niugini
22012,Adeuomphalus collinsi,1,1,1,152.100006,-3.800000,Papua Niugini
22015,Fucaria mystax,1,1,1,152.580002,-3.310000,Papua Niugini
22017,Lurifax japonicus,1,1,1,140.067001,31.466700,日本


In [45]:
data3_copy.isna().sum()

marine_species    0
presence          0
origin            0
seasonal          0
longitude         0
latitude          0
country           0
dtype: int64

In [46]:
data3_copy['country'].unique()

array(['México', 'United States', 'Guatemala', 'El Salvador', 'Honduras',
       'Costa Rica', 'Panamá', 'Colombia', 'France', 'Tonga', 'Sāmoa',
       'Kiribati', 'South Africa', 'Moçambique', 'السودان', 'السعودية',
       'Tanzania', 'Djibouti جيبوتي', 'Comores Komori جزر القمر',
       'Madagasikara / Madagascar', 'البحرين', 'Sesel',
       'Mauritius / Maurice', 'پاکستان', 'British Indian Ocean Territory',
       'India', 'ශ්\u200dරී ලංකාව இலங்கை', 'မြန်မာ', 'Australia',
       'Indonesia', 'ประเทศไทย', 'Malaysia', 'Việt Nam', 'Brunei', '臺灣',
       '中国', 'Philippines', '日本', 'Belau', 'Micronesia', 'Papua Niugini',
       'Solomon Islands', 'Vanuatu', 'Ṃajeḷ', 'Viti', 'Tuvalu',
       'বাংলাদেশ', 'Kenya', 'عمان', 'Singapore', 'Belize',
       'New Zealand/Aotearoa', 'Cuba', 'Türkiye', 'България', 'Россия',
       'الإمارات العربية المتحدة', 'Nicaragua', 'The Bahamas', 'Jamaica',
       'Ayiti', 'República Dominicana', 'Bermuda', 'Grenada',
       'Trinidad and Tobago', 'Saint Lucia

Replace the values in order to unify the values with those of the previous database for later merging

In [47]:
data3_copy = data3.copy()
data3_copy.rename(columns={'sci_name': 'marine_species'}, inplace=True)
data3_copy['country'] = data3_copy['country'].replace({'Perú': 'Peru', 'Sudán': 'Sudan', 'México': 'Mexico', 'Panamá': 'Panama', 'Việt Nam': 'Vietnam', 'Mauritius / Maurice': 'Mauritius','Madagasikara / Madagascar': 'Madagascar', 'Moçambique': 'Mozambique', 'NewZealand/Aotearoa': 'New Zealand', 'República Dominicana': 'Dominican Republic', 'Algérie': 'Algeria', 'Baréin': 'Bahrain', 'Brasil': 'Brazil', 'България': 'Bulgaria', 'Россия': 'Russia','Türkiye': 'Turkey', "السودان": "Sudán", "السعودية": "Arabia Saudita", 'Djibouti جيبوتي': 'Yibuti', 'ශ්‍රී ලංකාව இலங்கை': 'Sri Lanka', 'မြန်မာ': 'Myanmar / Birmania', 'ประเทศไทย': 'Thailand', '臺灣': 'Taiwan', '中国': 'China', '日本': 'Japon', 'বাংলাদেশ': 'Bangladesh', 'جزر القمر': 'Comoras', 'البحرين': 'Baréin', 'پاکستان': 'Pakistan', 'عمان': 'Oman', 'الإمارات العربية المتحدة': 'United Arab Emirates', 'Algérie / ⵍⵣⵣⴰⵢⴻⵔ / الجزائر': 'Algérie', 'Hrvatska': 'Croatia', 'Kūki ʻĀirani': 'Cook Islands', 'Comores Komori جزر القمر': 'Comores'})


In [48]:
data3_copy['country'].unique()

array(['Mexico', 'United States', 'Guatemala', 'El Salvador', 'Honduras',
       'Costa Rica', 'Panama', 'Colombia', nan, 'France', 'Tonga',
       'Sāmoa', 'Kiribati', 'South Africa', 'Mozambique', 'Sudán',
       'Arabia Saudita', 'Tanzania', 'Yibuti', 'Comores', 'Madagascar',
       'Baréin', 'Sesel', 'Mauritius', 'Pakistan',
       'British Indian Ocean Territory', 'India', 'Sri Lanka',
       'Myanmar / Birmania', 'Australia', 'Indonesia', 'Thailand',
       'Malaysia', 'Vietnam', 'Brunei', 'Taiwan', 'China', 'Philippines',
       'Japon', 'Belau', 'Micronesia', 'Papua Niugini', 'Solomon Islands',
       'Vanuatu', 'Ṃajeḷ', 'Viti', 'Tuvalu', 'Bangladesh', 'Kenya',
       'Oman', 'Singapore', 'Belize', 'New Zealand/Aotearoa', 'Cuba',
       'Turkey', 'Bulgaria', 'Russia', 'United Arab Emirates',
       'Nicaragua', 'The Bahamas', 'Jamaica', 'Ayiti',
       'Dominican Republic', 'Bermuda', 'Grenada', 'Trinidad and Tobago',
       'Saint Lucia', 'Barbados', 'Suriname', 'Cayman Island

Group by to know how many different marine species can be seen in each country

In [49]:
species_counts = data3_copy.groupby('country')['marine_species'].nunique().reset_index()

In [50]:
species_counts

,country,marine_species
0,Algérie,1
1,Antigua and Barbuda,1
2,Arabia Saudita,1
3,Australia,7
4,Ayiti,2
...,...,...
78,Venezuela,1
79,Vietnam,8
80,Viti,2
81,Yibuti,1


In [53]:
nomad = pd.read_csv('../Data/cleaned/nomad_cities.csv')

Merge both dataframes. Now, we can see for each city in which we have different values for digital nomads, how many differents species can be seen

In [54]:
marine_specie = nomad.merge(species_counts, on='country')

In [56]:
marine_specie.head()

,country,city,coffee_in_cafe,cost_beer,cost_coworking,cost_nomad,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,internet_speed,life_score,nightlife,nomad_score,places_to_work,racism,safety,marine_species
0,Thailand,Chiang Mai,0.85,0.85,98.88,777,0.8,78.8,0.6,0.2,0.6,14,0.75,0.40,0.95,0.8,0.40,0.80,7
1,Thailand,Phuket,1.41,1.41,155.43,1012,0.6,78.8,0.4,0.2,0.6,14,0.75,0.82,0.85,0.8,0.42,0.80,7
2,Thailand,Bangkok,2.12,2.12,131.41,1197,0.8,78.8,1.0,0.2,1.0,24,0.72,1.00,0.84,1.0,0.42,0.77,7
3,Thailand,Ko Samui,1.41,1.41,169.56,1352,0.8,78.8,0.4,0.2,1.0,15,0.80,0.80,0.84,0.8,0.40,0.80,7
4,Thailand,Chiang Rai,1.41,1.41,195.38,1134,0.6,78.8,0.6,0.2,1.0,12,0.76,0.80,0.74,0.6,0.60,0.60,7


In [57]:
marine_species = marine_specie.to_csv('../Data/cleaned/points_data.csv', index=False)

Upload the last database

In [58]:
data4 = pd.read_excel('../Data/raw/RB_Fatal_Accident_Database_100725.xls')
data4.head()

,№,Date,Deceased's Name,Rebreather involved,Electronics fitted to rebreather for PPo2 Monitoring or Control,"Type of dive (Solo, Group, Cave etc.)",Depth of accident (/deepest/planned ) (m),Location,Experience,Age,Root Cause using fault tree method,Disabling Injury using fault tree method
0,612,24 Sept 2022\nwithin week prior?,"Alexandre ""Ax"" Hache",JJ-CCR\nDSV,Shearwater eCCR controller with SW NERD HUD,Deep Wreck Dive?,Unknown,Corsica,Expert,Unknown,Scant Data,Scant Data\nHyperoxia and then drowning possibly
1,603,2022-08-25 00:00:00,Unknown\nItalian?,Unknown,Unknown,Deep Cave Dive,Unknown,"Croatia, cave",Unknown,Unknown,Scant Data,Scant Data
2,611,2022-08-21 00:00:00,Denier Alimov,Megalodon,ISC eCCR,Unknown,Unknown,"USA, NJ, Texas Tower",Unknown,55,Scant Data,DCI
3,610,2022-07-23 00:00:00,Unknown,Unknown rebreather,Unknown,Deep Wreck Dive,60m,"France, Cherbourg, Wreck of Leopardville",Unknown,49,Scant Data,Scant Data
4,609,2022-07-16 00:00:00,Andrzej Zalewski,ISC Meg eCCR\nGolen Gear BOV,ISC eCCR,Deep Lake dive,?/40m,"Poland, Lake Hańcza",Experienced,Unknown,Scant Data,Scant Data


Normalize the values of the column Root Cause using fault tree method, that corresponds to the cause of the death

In [59]:
data4['Root Cause using fault tree method'].unique()

array(['Scant Data', 'Training Issue', 'Faulty design',
       'Human error / PPO2 sensors fault', 'Fault', 'Scant Data ',
       'Rebreather Design Fault', 'Human error / Rebreather design fault',
       'Delibrate', '?', 'Human Error',
       'Rebreather design fault / Human error', 'Scant data',
       'Training issues',
       'Equipment Failure / Rebreather Design Fault / Human Error',
       'Unknown', 'Gas Booster Design Fault', 'Rebreather design fault',
       'Entanglement  / Exhausted SCR ', 'Entanglement', 'Human error',
       'Rebreather failure of buddy',
       'Reported as rebreather failure of O2 control',
       'Human error / Equipment issue (not rebreather)',
       'Explosive trauma', 'Human error/Rebreather issue',
       'Exceeded Performance Envelope / Human error / Rebreather Issue\n',
       'Traumatic Injury', 'Scant Date', 'General diving hazard',
       'Rebreather issue / Human error', 'Poor equipment cleaning',
       'Human error / Rebreather issue', 'U

Drop the rows with missing values

In [60]:
data4.dropna(subset=['Root Cause using fault tree method'], inplace=True)

In [61]:
data4['Root Cause using fault tree method'] = data4['Root Cause using fault tree method'].str.lower().str.strip().replace(' ', '')

In [62]:
data4 = data4[~data4['Root Cause using fault tree method'].isin(['scant data', 'scant date', '?', 'Unknown', 'dcs'])]

In [63]:
data4['Root Cause using fault tree method'].unique()

array(['training issue', 'faulty design',
       'human error / ppo2 sensors fault', 'fault',
       'rebreather design fault', 'human error / rebreather design fault',
       'delibrate', 'human error',
       'rebreather design fault / human error', 'training issues',
       'equipment failure / rebreather design fault / human error',
       'unknown', 'gas booster design fault',
       'entanglement  / exhausted scr', 'entanglement',
       'rebreather failure of buddy',
       'reported as rebreather failure of o2 control',
       'human error / equipment issue (not rebreather)',
       'explosive trauma', 'human error/rebreather issue',
       'exceeded performance envelope / human error / rebreather issue',
       'traumatic injury', 'general diving hazard',
       'rebreather issue / human error', 'poor equipment cleaning',
       'human error / rebreather issue', 'underlying illness',
       'rebreather failure/human error',
       'general diving hazard / entrapment', 'human e

Create a formula to normalize the most common values 

In [64]:
def normalize_causes(value):
    if any(keyword in str(value) for keyword in ['underlying illness', 'heart attack', 'cns oxygen toxicity', 'explosive trauma', 'traumatic injury']):
        return 'medical problems'
    elif any(keyword in str(value) for keyword in ['human', 'training issues', 'training issue', 'insufficient gas associated with panic']):
        return 'human error'
    elif any(keyword in str(value) for keyword in ['delibrate', 'rebreather', 'faulty design', 'rebreather design fault', 'gas booster design fault', 'closed oxygen cylinder', 'poor equipment cleaning']):
        return 'equipment problems'
    elif any(keyword in str(value) for keyword in ['entanglement  / exhausted scr', 'entanglement', 'general diving hazard / entrapment']):
        return 'entanglement'
    elif any(keyword in str(value) for keyword in ['fault', 'unknown']):
        return 'others'
    else:
        return value

In [65]:
data4['Root Cause using fault tree method'] = data4['Root Cause using fault tree method'].apply(normalize_causes)

In [66]:
data4['Root Cause using fault tree method'].unique()

array(['human error', 'equipment problems', 'others', 'entanglement',
       'medical problems', 'general diving hazard',
       'trapped in probable illegal fishing net',
       'exceeded performance envelope', 'marine life: bull shark.'],
      dtype=object)

In [67]:
data4['Root Cause using fault tree method'].value_counts()

Root Cause using fault tree method
human error                                127
equipment problems                          98
medical problems                            26
exceeded performance envelope                7
others                                       4
entanglement                                 3
general diving hazard                        3
trapped in probable illegal fishing net      1
marine life: bull shark.                     1
Name: count, dtype: int64

In [68]:
marine_species_copy = pd.read_csv('../Data/cleaned/points_data.csv')

In [69]:
marine_species_copy

,country,city,coffee_in_cafe,cost_beer,cost_coworking,cost_nomad,female_friendly,fragile_states_index,free_wifi_available,freedom_score,friendly_to_foreigners,internet_speed,life_score,nightlife,nomad_score,places_to_work,racism,safety,marine_species
0,Thailand,Chiang Mai,0.85,0.85,98.88,777,0.8,78.8,0.6,0.2,0.6,14,0.75,0.40,0.95,0.8,0.40,0.80,7
1,Thailand,Phuket,1.41,1.41,155.43,1012,0.6,78.8,0.4,0.2,0.6,14,0.75,0.82,0.85,0.8,0.42,0.80,7
2,Thailand,Bangkok,2.12,2.12,131.41,1197,0.8,78.8,1.0,0.2,1.0,24,0.72,1.00,0.84,1.0,0.42,0.77,7
3,Thailand,Ko Samui,1.41,1.41,169.56,1352,0.8,78.8,0.4,0.2,1.0,15,0.80,0.80,0.84,0.8,0.40,0.80,7
4,Thailand,Chiang Rai,1.41,1.41,195.38,1134,0.6,78.8,0.6,0.2,1.0,12,0.76,0.80,0.74,0.6,0.60,0.60,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,Belize,San Pedro,1.99,1.99,162.41,4228,0.6,66.0,0.4,0.6,0.8,3,0.68,1.00,0.47,0.8,0.00,0.60,5
765,Mozambique,Maputo,2.43,2.43,197.49,1965,0.4,87.8,0.4,0.4,0.8,2,0.53,0.40,0.47,0.4,0.60,0.60,4
766,Cayman Islands,George Town,2.65,2.65,150.00,4358,0.6,66.1,0.6,0.2,0.6,2,0.66,0.60,0.46,0.6,0.40,0.60,3
767,Madagascar,Antananarivo,1.18,1.18,255.30,1285,0.4,84.2,0.4,0.6,0.4,0,0.50,0.20,0.46,0.4,0.60,0.80,1


In [70]:
accidents = data4.copy()

In [71]:
accidents['Location'].unique()

array(['US, Lake Erie, ShipWreck', 'US, Idaho, Bear Lake State Park',
       'Austalia, Port of Newcastle, Bulk Carrier',
       'Portugal, NATO Exercise', 'Australia, Manly, Shelly Beach',
       'UK, Isles of Scilly', 'France, Bay of Cavalaire, Wreck of Togo',
       'France, Marcilhac-sur-Cele, Emergence of Ressel',
       'Israel, Palistine Gaza', 'US, Florida, Ginnie Springs',
       'Japan, Okinawa', 'UK, Cumbria, Lancaster Hole',
       'Thailand, Hong Song Cave', 'Greece, Britannic wreck',
       'France, Lot, Marchepied', 'UK, Plymouth, East Point WWI wreck',
       'France, north of Nice, System of La Mescla',
       'USA, Wisconsin, wreck of L.R.Doty', 'UK, Stoney Cove',
       'Poland, wreck Bremerhaven', 'Hawaii', 'UAE, wreck U533', 'UK',
       'USA', 'Australia, Melbourne, Blairgowrie Pier', 'Germany, ',
       'China, Hebei Province, Panjiakou, underwater section of ancient Great Wall ',
       'USA, Lake Superior, Judge Hart Wreck',
       'Cyprus, Larnaca, Green Bay',

In [72]:
accidents = accidents.drop(accidents[(accidents["Location"] == "Unknown") | (accidents["Location"] == "UAE, wreck U533") | (accidents["Location"] == "PNG") | (accidents["Location"] == "Not UK") | (accidents["Location"] == "Cocos") | (accidents["Location"] == "Burmuda, Tuckers Town") | (accidents["Location"] == "Black Sea, Crimea") | (accidents["Location"] == "-")].index)


In [73]:
accidents = accidents.rename(columns={'Location': 'country'})

Normalize alse the countries, as data is dirty

In [74]:
accidents = accidents.replace({'Austalia, Port of Newcastle, Bulk Carrier':'Australia',
'Australia, Manly, Shelly Beach':'Australia',
'Australia, Melbourne, Blairgowrie Pier':'Australia',
'Australia, Sydney, Palm Beach, ':'Australia',
'Australia':'Australia',
'Australia, Queenscliff, SS Rotamahana wreck':'Australia',
'Australia, Bermagui':'Australia',
'Australia, Sydney':'Australia',
'Australia ?':'Australia',
'Australia, Jervis Bay':'Australia',
'S. Australia, Port Noarlunga':'Australia',
'Austria, Weyregg am Attersee ':'Austria',
'Canada, Vancouver':'Canada',
'Canada, Tobermory, Ontario.':'Canada',
'Canada, Tobermory':'Canada',
'Canada, St Wreck of Lawrence Seaway, Roy A. Jodrey':'Canada',
'Canada, Hamilton, Ontario':'Canada',
'British Columbia, Port Hardy':'Canada',
'Canada':'Canada',
'Cayman Islands':'Cayman Islands',
'Chile':'Chile',
'China, Hebei Province, Panjiakou, underwater section of ancient Great Wall ':'China',
'Croatia':'Croatia',
'Croatia, Cavtat.':'Croatia',
'Curacao':'Curacao',
'Cyprus, Larnaca, Green Bay':'Cyprus',
'Cypress, Larnaca':'Cyprus',
'Denmark':'Denmark',
'Egypt, Sharm El Sheikh,':'Egypt',
'Egypt, Sharm El Sheikh':'Egypt',
'Egypt, Safaga':'Egypt',
'Egypt, Dahab, Red Sea':'Egypt',
'Egypt, Red Sea':'Egypt',
'Egypt':'Egypt',
'Finland, Benholmenin Karten':'Finland',
'Finland, Ojamo Mine':'Finland',
'Florida, Wakulla, SpringsState Park':'Florida',
'France, Bay of Cavalaire, Wreck of Togo':'France',
'France, Marcilhac-sur-Cele, Emergence of Ressel':'France',
'France, Lot, Marchepied':'France',
'France, north of Nice, System of La Mescla':'France',
'France, Seine Bay':'France',
'France, Font Estramar, a resurgence near Perpignan in the South of France':'France',
'France, Dragonnière Gaud':'France',
'France, Calaliere Le Lavandou':'France',
'France, Le Ressel':'France',
'France':'France',
'France, Source de Doubs':'France',
'France, Cavalaire, Rubis WWII Sub Wreck':'France',
'France, wreck Henkel Mediterranean':'France',
'France, La Londe Les Maures':'France',
'France, Becon les Granites':'France',
'France, Fabregas near Toulon':'France',
'France, Saint Andeol':'France',
"France, Cote d'Azur":'France',
'Germany, ':'Germany',
'Germany, Hemmoor':'Germany',
'Germany, Wildeshutz':'Germany',
'Germany':'Germany',
'Germany, Bigge-See':'Germany',
'Greece, Britannic wreck':'Greece',
'Greece, Arcadia (Peloponnes), "Sintzi" cave':'Greece',
'Greece, Britannic':'Greece',
'Greece, Cape Krios, Crete':'Greece',
'Greece':'Greece',
'Hawaii':'Hawaii',
'Holland, Freisan Islands, Schiermonnikoog':'Holland',
'Indonesia, Seven Skies Wreck':'Indonesia',
'Indonesia, Batam Island':'Indonesia',
'Ireland':'Ireland',
'Ireland, Donegal':'Ireland',
'Ireland, Co. Wexford':'Ireland',
'Ireland, Castletownshend, Co Cork':'Ireland',
'Israel, Palistine Gaza':'Israel',
'Israel, Ashdod Port':'Israel',
'Israel, Eilat (Red Sea)':'Israel',
'Isreal, Eliat, Close to Underwater Marine Observatory':'Israel',
'Italy, Lake Iseo':'Italy',
'Italy, Sicily, Ustica':'Italy',
'Italy, Lake Garda':'Italy',
'Italy, Miramare':'Italy',
'Italy, Lake Iseo, in Tavernola':'Italy',
'Italy, Porto Santo Stefano':'Italy',
'Italy, Cornino':'Italy',
'Italy, Garda Lake':'Italy',
'Italy, Lago albano di castelgandolfoo (Lake Albano)':'Italy',
'Italy, Lake Como':'Italy',
'Italy, Tuscany, Pollaccia Cave':'Italy',
'Italy, Marano':'Italy',
'Italy':'Italy',
'Japan, Okinawa':'Japan',
'Japan, Numazu, Shizuoka Prefecture':'Japan',
'Japan, Ida beach, Numazu':'Japan',
'Mexico, Cenote Jailhouse/Sistema Ox Bel Ha':'Mexico',
'Mexico, San Augustin, Huautla Cave System':'Mexico',
'Micronesia, Truk lagoon':'Micronesia',
'Micronesia. Truk lagoon':'Micronesia',
'Netherlands, Bonaire':'Netherlands',
'New Guinea':'New Guinea',
'New Zealand, Ship Cove':'New Zealand',
'New Zealand':'New Zealand',
'North Pole, Expedition':'North Pole',
'Norway, Bremanger':'Norway',
'Norway, Nauntes, Øygarden':'Norway',
'Norway, Plura cave':'Norway',
'Norway, Håkøy, Tirptitz wreck':'Norway',
'Norway,  Askøy, Bergen.':'Norway',
'Norway':'Norway',
'Philippines, Puerto Galera, LaLaguna Beach resort':'Philippines',
'Philippines, Panagsama, Moalboal':'Philippines',
'Philippines, Olongapo, Subic Bay.':'Philippines',
'Philippines':'Philippines',
'Poland, wreck Bremerhaven':'Poland',
'Poland, Hancza':'Poland',
'Poland,  Darlowo (Baltic Sea)':'Poland',
'Poland, Quarry Jaworzno Szczakowa':'Poland',
'Portugal, NATO Exercise':'Portugal',
'Portugal, Setubal':'Portugal',
'Russia, Blue Lake, in Kabardino-Balkaria':'Russia',
'Russia, Baltic sea, Finnish Gulf':'Russia',
'Singapore':'Singapore',
'South Africa, Sodwana':'South Africa',
'South Africa, Jesser Canyon, Sodwana Bay':'South Africa',
'South Africa, Hermanus':'South Africa',
'S. Africa, Boesmansgat ':'South Africa',
'South East Asia':'South East Asia',
'Spain, Hondarribia':'Spain',
'Spain':'Spain',
'Sweden, West Coast':'Sweden',
'Switzerland, Rossinière':'Switzerland',
'Switzerland, Lake Neuenburg, Abyss':'Switzerland',
'Switzerland':'Switzerland',
'Switzerland, Zugersee (Lake of Zug).':'Switzerland',
'Switzerland, Neuenburger See.':'Switzerland',
'Thailand, Hong Song Cave':'Thailand',
'Thailand, Khao Sok area, Surat Thani, Chiewlan Reservoir, Tiger Cave':'Thailand',
'Thailand, Song Hong Cave':'Thailand',
'Thailand, China Sea, Unidentified wreck close to the Tottorri Maru':'Thailand',
'Thailand, Samran Pinnacle (Latitude N 90 degrees, 57’, 56.8”, Longitude E 99 degrees, 55’, 13.1”) located between Koh Tao and Koh Hin Bai':'Thailand',
'Thailand':'Thailand',
'UK, Isles of Scilly':'UK',
'UK, Cumbria, Lancaster Hole':'UK',
'UK, Plymouth, East Point WWI wreck':'UK',
'UK, Stoney Cove':'UK',
'UK':'UK',
'UK, Swansea, Kyarra and Aparima wrecks':'UK',
'UK, Eastbourne, Mid Channel':'UK',
"UK, Jersey, Shore Dive, St Catherine's Breakwater":'UK',
'UK, Scotland, Cape Wrath':'UK',
'UK, Brixham':'UK',
'UK, Scotland, Orkney, Westray':'UK',
'UK, Scotland, Scapa Flow, Mark Graff':'UK',
'UK, England, Dorset':'UK',
'UK, Swithland Woods':'UK',
'UK, Dorothea quarry':'UK',
'UK, Cornwall Quarry':'UK',
'UK, England, Littlehampton':'UK',
'UK, Isle of Man':'UK',
'UK,England,':'UK',
'UK, Chepstow':'UK',
'UK, Channel Islands, Sark':'UK',
'UK, Scotland, Loch':'UK',
'UK,England, Plymouth':'UK',
'UK, Salsette, Portland':'UK',
'UK, England, Isle of Wight':'UK',
'UK, England, South Coast, Portland':'UK',
'UK, Northern Ireland, Ardglass':'UK',
'UK,England, Devon, SS Afric':'UK',
'UK, Mineries Pool, Somerset':'UK',
'UK, Wookey Hole Cave, Somerset':'UK',
'US, Lake Erie, ShipWreck':'USA',
'US, Idaho, Bear Lake State Park':'USA',
'US, Florida, Ginnie Springs':'USA',
'USA, Wisconsin, wreck of L.R.Doty':'USA',
'USA':'USA',
'USA, Lake Superior, Judge Hart Wreck':'USA',
'USA, Florida Keys, Queen of Nassau':'USA',
'USA, Florida, Blue Grotto':'USA',
'USA, Florida, Ginnie Springs Park':'USA',
'USA, WA, Mukilteo, Lighthouse Park':'USA',
'USA, Gibsonburg, Ohio, Sandusky County Park, White Star Quarry':'USA',
'USA, Woodvolle Karst Plain, Whisky Still Sink':'USA',
'USA, Hawaii':'USA',
'USA, Florida, Marathon':'USA',
'USA, Key West, Oriskany Wreck':'USA',
'USA, Key West, Vandenburg Wreck ':'USA',
"USA, Eagle's Nest cave":'USA',
'USA, Florida, Deerfield Beach':'USA',
'USA, Lake Michigan':'USA',
'USA, MA, wreck Poling':'USA',
'USA, N.J. Bargegat Inlet':'USA',
'USA, Key Largo':'USA',
'USA, Jackson Blue Cave':'USA',
'USA, North Carolina, Atlantic Ocean':'USA',
'USA, Andrea Doria':'USA',
'USA, Wisconsin, Wazi Reservoir':'USA',
'USA, Florida, Pompano Bch':'USA',
"USA, Devil's system, Ginnie Springs,":'USA',
'USA, NJ?':'USA',
'USA, Florida':'USA',
'USA, Seattle':'USA',
'USA, San Mateo, CA':'USA',
'USA, Los Angeles':'USA',
'USA, Louise Pit, MN':'USA',
'USA, San Pedro, Olympic wreck':'USA',
'USA, Lake Norman Quarry':'USA',
'USA, Washington':'USA',
'USA, New Jersey, Bald Eagle wreck':'USA',
'USA, PA':'USA',
'USA, Bethlehem PA':'USA',
'USA, Bonne Terre Mine, MN':'USA',
'USA, Willow Springs Quarry, Richland, PA':'USA',
'USA, Bainbridge, PA':'USA',
'USA, Mukilteo, WA':'USA',
'USA, La Jolla Shores, CA':'USA',
'USA, Friday Harbor, WA':'USA',
'USA, San Clemente Island, CA':'USA',
'Nantucket harbour, US':'USA',
'Wales, Holyhead':'Wales',})

In [75]:
accidents['country'].unique()


array(['USA', 'Australia', 'Portugal', 'UK', 'France', 'Israel', 'Japan',
       'Thailand', 'Greece', 'Poland', 'Hawaii', 'Germany', 'China',
       'Cyprus', 'Micronesia', 'Curacao', 'Egypt', 'Italy',
       'South East Asia', 'Finland', 'Mexico', 'Chile', 'Norway',
       'Netherlands', 'Switzerland', 'Russia', 'Denmark', 'Indonesia',
       'Philippines', 'Canada', 'New Zealand', 'Croatia', 'Ireland',
       'Spain', 'South Africa', 'Sweden', 'Austria', 'Wales', 'Holland',
       'Florida', 'Singapore', 'North Pole', 'New Guinea',
       'Cayman Islands'], dtype=object)

In [76]:
accidents

,№,Date,Deceased's Name,Rebreather involved,Electronics fitted to rebreather for PPo2 Monitoring or Control,"Type of dive (Solo, Group, Cave etc.)",Depth of accident (/deepest/planned ) (m),country,Experience,Age,Root Cause using fault tree method,Disabling Injury using fault tree method
7,606,2022-06-12 00:00:00,Tyler Cullinan,Hollis Prism 2,Shearwater eCCR controller,Trimix Training dive,60m,USA,Novice (at this depth),31,human error,Scant Data
8,605,2022-05-23 00:00:00,Kyle Walker,SubGravity Defender,Subgravity/IQSub,Novice Dive,1.5m,USA,Novice,31,human error,Hypoxia
9,604,2022-05-09 00:00:00,Bruna Borges,Unconfirmed rebreather type,Unconfirmed,Drug Smuggling,<10m,Australia,Unknown (on CCR),31,human error,Scant Data
15,598,predates Sept 2021\nExact day/month/year unknown,Belgium Military Diver,SCR,Divers had removed SCRs PPO2 monitor due to hi...,MCM Training dive,74m,Portugal,Expert,Unknown,equipment problems,Hyperoxic
18,595,2021-08-05 00:00:00,Stephan Ho,GUE JJ-CCR,Shearwater eCCR controller,"Solo dive, effectively; with freediving buddy",~4m,Australia,Expert,Unknown,human error,Hypoxia\n
...,...,...,...,...,...,...,...,...,...,...,...,...
605,8,1967-08-10 00:00:00,Able-Seaman Clearance Diver Jeffrey Thomas Hales,O2 Rebreather,RAN Issue,Combat Swimming,0m/8m,Australia,Expert,19,human error,Hyperoxia / Hypoxia
607,6,1962-11-03 00:00:00,"E J ""Jack"" Waddon",Homemade rebreather,NaN,Cave Diving,3,UK,Experienced,30,equipment problems,"Hypoxia, though cause of death was pulmonary o..."
608,5,1956-08-01 00:00:00,William Edgerton,Lambertson,Unknown,Training dive,3m,USA,Novice,23,equipment problems,Hypoxia.
610,3,1951-12-28 00:00:00,John Justin Williams,WWII O2 CCR,NaN,Solo Diving,3m,Australia,Novice,33,human error,Hypoxia due to O2 exhausted


In [77]:
causes = accidents.groupby('country')['Root Cause using fault tree method'].agg(lambda x: ', '.join(sorted(set(x)))).reset_index()



In [78]:
causes

,country,Root Cause using fault tree method
0,Australia,"equipment problems, human error"
1,Austria,medical problems
2,Canada,"equipment problems, human error, medical problems"
3,Cayman Islands,human error
4,Chile,medical problems
5,China,human error
6,Croatia,"human error, medical problems"
7,Curacao,equipment problems
8,Cyprus,human error
9,Denmark,human error


In [79]:
grouped = accidents.groupby(['country', 'Root Cause using fault tree method']).size().reset_index(name='count')

result = grouped.groupby('country').agg({'Root Cause using fault tree method': ', '.join, 'count': sum}).reset_index()



Join the databases 

In [80]:
accidents = pd.merge(accidents, result, on='country', how='left')


In [81]:
accidents

,№,Date,Deceased's Name,Rebreather involved,Electronics fitted to rebreather for PPo2 Monitoring or Control,"Type of dive (Solo, Group, Cave etc.)",Depth of accident (/deepest/planned ) (m),country,Experience,Age,Root Cause using fault tree method_x,Disabling Injury using fault tree method,Root Cause using fault tree method_y,count
0,606,2022-06-12 00:00:00,Tyler Cullinan,Hollis Prism 2,Shearwater eCCR controller,Trimix Training dive,60m,USA,Novice (at this depth),31,human error,Scant Data,"equipment problems, exceeded performance envel...",60
1,605,2022-05-23 00:00:00,Kyle Walker,SubGravity Defender,Subgravity/IQSub,Novice Dive,1.5m,USA,Novice,31,human error,Hypoxia,"equipment problems, exceeded performance envel...",60
2,604,2022-05-09 00:00:00,Bruna Borges,Unconfirmed rebreather type,Unconfirmed,Drug Smuggling,<10m,Australia,Unknown (on CCR),31,human error,Scant Data,"equipment problems, human error",13
3,598,predates Sept 2021\nExact day/month/year unknown,Belgium Military Diver,SCR,Divers had removed SCRs PPO2 monitor due to hi...,MCM Training dive,74m,Portugal,Expert,Unknown,equipment problems,Hyperoxic,"equipment problems, human error",2
4,595,2021-08-05 00:00:00,Stephan Ho,GUE JJ-CCR,Shearwater eCCR controller,"Solo dive, effectively; with freediving buddy",~4m,Australia,Expert,Unknown,human error,Hypoxia\n,"equipment problems, human error",13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,8,1967-08-10 00:00:00,Able-Seaman Clearance Diver Jeffrey Thomas Hales,O2 Rebreather,RAN Issue,Combat Swimming,0m/8m,Australia,Expert,19,human error,Hyperoxia / Hypoxia,"equipment problems, human error",13
257,6,1962-11-03 00:00:00,"E J ""Jack"" Waddon",Homemade rebreather,NaN,Cave Diving,3,UK,Experienced,30,equipment problems,"Hypoxia, though cause of death was pulmonary o...","equipment problems, exceeded performance envel...",37
258,5,1956-08-01 00:00:00,William Edgerton,Lambertson,Unknown,Training dive,3m,USA,Novice,23,equipment problems,Hypoxia.,"equipment problems, exceeded performance envel...",60
259,3,1951-12-28 00:00:00,John Justin Williams,WWII O2 CCR,NaN,Solo Diving,3m,Australia,Novice,33,human error,Hypoxia due to O2 exhausted,"equipment problems, human error",13


In [82]:
causes

,country,Root Cause using fault tree method
0,Australia,"equipment problems, human error"
1,Austria,medical problems
2,Canada,"equipment problems, human error, medical problems"
3,Cayman Islands,human error
4,Chile,medical problems
5,China,human error
6,Croatia,"human error, medical problems"
7,Curacao,equipment problems
8,Cyprus,human error
9,Denmark,human error


In [83]:
merged_df = accidents.merge(causes)


In [84]:
merged_df

,№,Date,Deceased's Name,Rebreather involved,Electronics fitted to rebreather for PPo2 Monitoring or Control,"Type of dive (Solo, Group, Cave etc.)",Depth of accident (/deepest/planned ) (m),country,Experience,Age,Root Cause using fault tree method_x,Disabling Injury using fault tree method,Root Cause using fault tree method_y,count,Root Cause using fault tree method
0,606,2022-06-12 00:00:00,Tyler Cullinan,Hollis Prism 2,Shearwater eCCR controller,Trimix Training dive,60m,USA,Novice (at this depth),31,human error,Scant Data,"equipment problems, exceeded performance envel...",60,"equipment problems, exceeded performance envel..."
1,605,2022-05-23 00:00:00,Kyle Walker,SubGravity Defender,Subgravity/IQSub,Novice Dive,1.5m,USA,Novice,31,human error,Hypoxia,"equipment problems, exceeded performance envel...",60,"equipment problems, exceeded performance envel..."
2,576,2021-01-21 00:00:00,Thomas Brian Ellis,KISS SideWinder,Shearwater,Solo Cave Dive,Unknown,USA,Expert,59,equipment problems,Hypoxia,"equipment problems, exceeded performance envel...",60,"equipment problems, exceeded performance envel..."
3,539,2019-06-28 00:00:00,Jim Winn,Mares rEvo eCCR,Shearwater eCCR controller,Buddy DPV deep wreck cold water dive,90m\n@4'C,USA,Novice (at this depth),53,human error,Hypoxia/Hypercapnia,"equipment problems, exceeded performance envel...",60,"equipment problems, exceeded performance envel..."
4,538,2019-06-28 00:00:00,Susan Winn,Mares rEvo eCCR,Shearwater eCCR controller,Buddy DPV deep wreck cold water dive,90m\n@4'C,USA,Novice (at this depth),53,human error,Hypoxia,"equipment problems, exceeded performance envel...",60,"equipment problems, exceeded performance envel..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,40,1999-02-15 00:00:00,Henry Kendall,Cis-Lunar Mk5,Cis-Lunar,Solo Diving,3m,Florida,Expert,72,equipment problems,Hypoxia,equipment problems,1,equipment problems
257,38,1999-01-01 00:00:00,Unknown,Drager FGT 1/D,Nil,Buddy Diving,40m,Singapore,NaN,NaN,equipment problems,Caustic Cocktail / Drowning / Hypercapnia,equipment problems,1,equipment problems
258,34,1998-07-01 00:00:00,Andrey Rodjkov,IDA,Unknown,Ice Diving,Unknown,North Pole,Experienced,?,exceeded performance envelope,Inhale counterlung or the inhale mushroom valv...,exceeded performance envelope,1,exceeded performance envelope
259,28,1998-04-04 00:00:00,Harald Kvam,Drager Dolphin,Unknown,Scant data,29m,New Guinea,-,NaN,human error,Hyperoxia from use of wrong mix.,human error,1,human error


In [85]:
merged_data = pd.merge(accidents, marine_species_copy, on="country")

In [86]:
merged_data.columns

Index(['№', 'Date', 'Deceased's Name', 'Rebreather involved',
       'Electronics fitted to rebreather for PPo2 Monitoring or Control',
       'Type of dive (Solo, Group, Cave etc.)',
       'Depth of accident (/deepest/planned ) (m)', 'country', 'Experience',
       'Age', 'Root Cause using fault tree method_x',
       'Disabling Injury using fault tree method',
       'Root Cause using fault tree method_y', 'count', 'city',
       'coffee_in_cafe', 'cost_beer', 'cost_coworking', 'cost_nomad',
       'female_friendly', 'fragile_states_index', 'free_wifi_available',
       'freedom_score', 'friendly_to_foreigners', 'internet_speed',
       'life_score', 'nightlife', 'nomad_score', 'places_to_work', 'racism',
       'safety', 'marine_species'],
      dtype='object')

In [87]:
merged_data = merged_data.drop(['Electronics fitted to rebreather for PPo2 Monitoring or Control', '№', 'Date', "Deceased's Name", 'Rebreather involved', 'Rebreather involved', 'Type of dive (Solo, Group, Cave etc.)', 'Experience', 'Age', 'Depth of accident (/deepest/planned ) (m)', 'Disabling Injury using fault tree method'], axis=1)
merged_data.head()

,country,Root Cause using fault tree method_x,Root Cause using fault tree method_y,count,city,coffee_in_cafe,cost_beer,cost_coworking,cost_nomad,female_friendly,...,freedom_score,friendly_to_foreigners,internet_speed,life_score,nightlife,nomad_score,places_to_work,racism,safety,marine_species
0,Australia,human error,"equipment problems, human error",13,Portland,5.00,5.00,300.00,3503,0.8,...,0.6,1.0,23,0.95,1.0,0.90,1.0,0.8,0.8,7
1,Australia,human error,"equipment problems, human error",13,Perth,3.02,3.02,332.05,1938,1.0,...,0.8,1.0,5,0.88,0.6,0.83,0.8,0.8,1.0,7
2,Australia,human error,"equipment problems, human error",13,Richmond,5.00,5.00,250.00,2824,0.8,...,0.6,0.6,24,0.84,0.8,0.81,0.8,0.8,0.6,7
3,Australia,human error,"equipment problems, human error",13,Richmond,5.00,5.00,250.00,2824,0.8,...,0.6,0.6,24,0.84,0.8,0.81,0.8,0.8,0.6,7
4,Australia,human error,"equipment problems, human error",13,Wollongong,5.66,5.66,226.40,2561,1.0,...,0.8,1.0,50,0.87,0.6,0.80,0.8,0.8,0.8,7


In [88]:
merged_data = merged_data.drop(['Root Cause using fault tree method_x'], axis=1)
merged_data.head()

,country,Root Cause using fault tree method_y,count,city,coffee_in_cafe,cost_beer,cost_coworking,cost_nomad,female_friendly,fragile_states_index,...,freedom_score,friendly_to_foreigners,internet_speed,life_score,nightlife,nomad_score,places_to_work,racism,safety,marine_species
0,Australia,"equipment problems, human error",13,Portland,5.00,5.00,300.00,3503,0.8,34.0,...,0.6,1.0,23,0.95,1.0,0.90,1.0,0.8,0.8,7
1,Australia,"equipment problems, human error",13,Perth,3.02,3.02,332.05,1938,1.0,22.5,...,0.8,1.0,5,0.88,0.6,0.83,0.8,0.8,1.0,7
2,Australia,"equipment problems, human error",13,Richmond,5.00,5.00,250.00,2824,0.8,34.0,...,0.6,0.6,24,0.84,0.8,0.81,0.8,0.8,0.6,7
3,Australia,"equipment problems, human error",13,Richmond,5.00,5.00,250.00,2824,0.8,34.0,...,0.6,0.6,24,0.84,0.8,0.81,0.8,0.8,0.6,7
4,Australia,"equipment problems, human error",13,Wollongong,5.66,5.66,226.40,2561,1.0,22.5,...,0.8,1.0,50,0.87,0.6,0.80,0.8,0.8,0.8,7


In [89]:
merged_data = merged_data.rename(columns={'Root Cause using fault tree method_y': 'cause_of_death', 'count': 'number_of_deaths'})

In [90]:
merged_data

,country,cause_of_death,number_of_deaths,city,coffee_in_cafe,cost_beer,cost_coworking,cost_nomad,female_friendly,fragile_states_index,...,freedom_score,friendly_to_foreigners,internet_speed,life_score,nightlife,nomad_score,places_to_work,racism,safety,marine_species
0,Australia,"equipment problems, human error",13,Portland,5.00,5.00,300.00,3503,0.8,34.0,...,0.6,1.00,23,0.95,1.0,0.90,1.0,0.8,0.8,7
1,Australia,"equipment problems, human error",13,Perth,3.02,3.02,332.05,1938,1.0,22.5,...,0.8,1.00,5,0.88,0.6,0.83,0.8,0.8,1.0,7
2,Australia,"equipment problems, human error",13,Richmond,5.00,5.00,250.00,2824,0.8,34.0,...,0.6,0.60,24,0.84,0.8,0.81,0.8,0.8,0.6,7
3,Australia,"equipment problems, human error",13,Richmond,5.00,5.00,250.00,2824,0.8,34.0,...,0.6,0.60,24,0.84,0.8,0.81,0.8,0.8,0.6,7
4,Australia,"equipment problems, human error",13,Wollongong,5.66,5.66,226.40,2561,1.0,22.5,...,0.8,1.00,50,0.87,0.6,0.80,0.8,0.8,0.8,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,South Africa,"equipment problems, human error",4,Port Elizabeth,2.79,2.79,227.12,2369,0.6,69.9,...,0.6,0.80,2,0.61,0.4,0.53,0.6,0.6,0.4,4
1112,South Africa,"equipment problems, human error",4,Durban,4.19,4.19,244.31,3240,0.6,69.9,...,0.6,0.80,3,0.59,0.4,0.49,0.4,0.6,0.4,4
1113,South Africa,"equipment problems, human error",4,Johannesburg,2.79,2.79,195.45,2467,0.4,69.9,...,0.6,0.80,4,0.42,0.4,0.35,0.8,0.6,0.2,4
1114,Singapore,equipment problems,1,Singapore,2.38,2.38,212.16,3765,1.0,32.9,...,0.2,0.62,55,0.86,1.0,0.78,0.6,0.8,1.0,5


In [91]:
merged_data = merged_data.to_excel('../Data/cleaned/RB_Fatal_Accident_Database_100725.xlsx', index=False)
merged_data